# Notebook M06 - Plume problem

This is an implementation of a 2D viscous flow code. The energy equation (for temperature) is solved at the cell centers (unlike the example in the chapter 10 notebook, which is solved at the basic nodes.

This notebook puts all of the major function definitions into separate source files and contains only the high-level program code.

In [1]:
# Import necessary packages
using SparseArrays
using LinearAlgebra
using IterativeSolvers
using WriteVTK
using Printf
using Statistics 
include("Grid.jl")
include("GridOperations.jl")
include("Markers.jl")
include("Stokes.jl")
include("Temperature.jl")

# note that we import pyplot last to avoid a name conflict with grid.
using PyPlot
include("Visualization.jl")

plots (generic function with 1 method)

# Problem setup

- Lithosphere with initial geotherm set by plate cooling

In [89]:
# problem set-up
using SpecialFunctions
# function to define plate cooling solution
function plate_cooling(Tsurf,Tbtm,yL0,kappa,y,t)
   # y is a vector of y coordinates where we need the solution
    T = Tsurf .+ (Tbtm-Tsurf) .* (y/yL0)
    for n in 1:20 # note 20 is an index in a summation from 1 to Inf
        T += (Tbtm-Tsurf)*2/pi .* 1/n*exp(-kappa*n^2*pi^2*t/yL0^2)*sin(n*pi*y/yL0)
    end
    return T
end

function halfspace_cooling(Tsurf,Tmantle,kappa,y,t)
    if t == 0.0
        if y==0.0
            return Tsurf
        else
            return Tmantle
        end
    else
        T = Tsurf + (Tmantle-Tsurf)*erf(y/2/sqrt(kappa*t))         
        return T
    end
end

function halfspace_cooling_from_thickness(Tsurf,Tmantle,kappa,y,thickness)
    t = (thickness/2)^2/kappa
    return halfspace_cooling(Tsurf,Tmantle,kappa,y,t)
end

# function to compute solidus and liquidus

# function to compute melt fraction

# function to compute viscosity
function viscosity(eta0::Float64,T::Float64,E::Float64 ; visc_max=1.0e25)
   # E should be given in J/mol/K
   # Expect all temperatures in kelvin.
   Tref = 1350.0+273.0
   R = 8.314 #J/mol/K
   viscosity = eta0*exp(E/R/Tref*(Tref/T-1))
   if viscosity > visc_max
      viscosity = visc_max
   end
   return viscosity
end

struct Materials
    # 1 - ambient mantle
    # 2 - eclogite
    alpha::Vector{Float64}
    rho0::Vector{Float64}
    Hr::Vector{Float64}
    Cp::Vector{Float64}
    kThermal::Vector{Float64}
    eta::Vector{Float64}
    Ea::Vector{Float64}
    function Materials()
         new([3e-5,3e-5],[3300.,3300.],[0.0,0.0],[1000.,1000.],[3.0,3.0],[1e21,1e21],[3e5,3e5])
    end
end

function update_marker_properties!(markers::Markers,materials::Materials)
    rho = markers.scalarFields["rho"]
    T = markers.scalarFields["T"]
    mmat = markers.integers[markers.integerFields["material"],:]
    eta = markers.scalarFields["eta"]
    for i in 1:markers.nmark                
        # re-compute density using the current temperature value
        # assume reference temperature is 273.0
        # markers.scalars[rho,i] = materials.rho0[mmat[i]] # don't update density - for comparison with gerya
        markers.scalars[rho,i] = materials.rho0[mmat[i]]*(1.0-materials.alpha[mmat[i]]*(markers.scalars[T,i]-273.0)) 
        markers.scalars[eta,i] = viscosity(materials.eta[mmat[i]],markers.scalars[T,i],materials.Ea[mmat[i]])
    end
end

function initial_conditions!(markers::Markers,materials::Materials)
    # Define geometric properties
    lithosphere_thickness = 1.5e5
    mantle_temperature = 1350.0
    
    material = markers.integerFields["material"]
    T = markers.scalarFields["T"]
    eta = markers.scalarFields["eta"]
    alpha = markers.scalarFields["alpha"]
    cp = markers.scalarFields["Cp"]
    Hr = markers.scalarFields["Hr"]
    for i in 1:markers.nmark
        mx = markers.x[1,i]
        my = markers.x[2,i]
        mr = ((mx-0)^2 + (my-2.85e6)^2)^0.5 
        
        # define initial cmb hot layer geometry
        h = 150e3 + (113e3-150e3)*(mx-0.0)/5e5
                
        # set material - eclogite at cmb
        if my > 2.85e6-h
           markers.integers[material,i] = 2
        else
           markers.integers[material,i] = 1
        end
                
        if my < lithosphere_thickness
            markers.scalars[T,i] = plate_cooling(273.0,1350.0+273.0,1.5e5,1e-6,my,100e6*3.15e7)
        else
            markers.scalars[T,i] = halfspace_cooling_from_thickness(2500.0+273.0,1350.0+273.0,1e-6,2.85e6-my,h)
        end
        ind = markers.integers[material,i]
        markers.scalars[eta,i] = viscosity(materials.eta[ind],markers.scalars[T,i],materials.Ea[ind])
        markers.scalars[alpha,i] = materials.alpha[ind]            
        markers.scalars[cp,i] = materials.Cp[ind]  
        markers.scalars[Hr,i] = materials.Hr[ind]  
    end
    update_marker_density!(markers,materials)
end

initial_conditions! (generic function with 1 method)

In [90]:
# plotting routine
function plume_plots(grid::CartesianGrid,vx,vy,rho,T)
    # make some diagnostic plots using pyplot
    fig, (ax1,ax2,ax3,ax4) = subplots(2,2)
    ax1.pcolor(grid.xc,grid.yc,rho)
    ax1.invert_yaxis()
    ax1.set_title("Density")
    ax2.pcolor(grid.xc,grid.yc,T)
    ax2.invert_yaxis()
    ax2.set_title("Temperature")
    ax3.pcolor(grid.xc,grid.yc,vx)
    ax3.invert_yaxis()
    ax3.set_title("vx")
    ax4.pcolor(grid.xc,grid.yc,vy)
    ax4.invert_yaxis()
    ax4.set_title("vy")
    
    fig.show()
end


plume_plots (generic function with 1 method)

# i2vis procedure

0. Set up the Eulerian grid and Lagrangian markers. Distribute the markers within the domain and assign initial conditions.
1. Compute density and viscosity at the basic nodes. Compute cell center viscosity via averaging.
2. Assemble and solve the Stokes equations.
3. Compute the maximum timestep from the velocity field and CFL number.
3. Transfer temperature from markers to basic nodes.
4. Compute the new temperature solution.
5. If the maximum temperature change exceeds a specified value, reduce the timstep and repeat previous step.
6. Compute the subgrid temperature changes and update the temperature on the markers.
7. Move the markers using 4th order Runge-Kutta

TO DO:
1. Calculate rhox (rho at vx points) -> (optionally) pass to Stokes.jl (Max)
3. Add free-surface stabilization terms to Stokes.jl (Lucy)
4. Add functionality for shear heating (Max)
5. Add functionality for adiabatic heating (Uthkarsh)

In [ ]:
function plume_model(nstep::Int64)
# nstep=1
    nx = 51
    ny = 285
    W = 5e5
    H = 2.850e6
    gx = 0.0
    gy = 10.0

    Tbcval = [0.0,0.0,273.0,2500.0+273.0]
    bc = BoundaryConditions(0,0,0,0) # currently does nothing but is required argument to stokes solver.
    materials = Materials()
    
    markx = 6
    marky = 6
    seconds_in_year = 3.15e7
    plot_interval = 1e6*seconds_in_year # plot interval in seconds
    end_time = 3e7*seconds_in_year
    dtmax = plot_interval
    grid = CartesianGrid(W,H,nx,ny)
    println("Creating Markers...")
    @time markers = Markers(grid,["alpha","Cp","T","rho","eta","Hr"],["material"] ; nmx=markx,nmy=marky,random=false)
    println("Initial condition...")
    @time initial_conditions!(markers, materials)

    # define arrays for k, rho, cp, H at the basic nodes. Fill them with constant values for now.
    kThermal = 3.0 .*ones(grid.ny,grid.nx);

    time = 0.0
    iout=0
    last_plot = 0.0
    rho_c = nothing
    dt = 1e10

      local rho_c
      local rho_vx
      local rho_vy
      local alpha
      local Hr
      local Cp_c
      local eta_s
      local eta_n
      local vxc
      local vyc
      local T
      local dt
      local dTmax
      local dTemp
      local Tnew
      local Tlast

     output_dir = "plume"
     itime=1
     for itime in 1:nstep
        update_marker_properties!(markers,materials)
        # 1. Transfer properties markers -> nodes
        # 1a. Basic Nodes
        eta_s_new, = marker_to_basic_node(markers,grid,["eta",]);
        # 1b. Cell Centers
        rho_c_new,Cp_c_new,alpha_new,eta_n_new,Tlast_new,Hr_new = marker_to_cell_center(markers,grid,["rho","Cp","alpha","eta","T","Hr"])
        # 1c. Vx and Vy nodes:
        rho_vx_new, = marker_to_stag(markers,grid,["rho",],"vx")
        rho_vy_new, = marker_to_stag(markers,grid,["rho",],"vy")
        
        # deal with any NaN values from interpolation:
        if itime > 1
            if any(isnan.(eta_s_new))
                println("found nan values")
            end
            replace_nan!(eta_s,eta_s_new)
            replace_nan!(rho_c,rho_c_new)
            replace_nan!(Hr,Hr_new)
            replace_nan!(Cp_c,Cp_c_new)
            replace_nan!(alpha,alpha_new)
            replace_nan!(eta_n,eta_n_new)
            replace_nan!(Tlast,Tlast_new)
            replace_nan!(rho_vx,rho_vx_new)
            replace_nan!(rho_vy,rho_vy_new)
        end
        # Copy field data 
        rho_vx = copy(rho_vx_new)
        rho_vy = copy(rho_vy_new)
        rho_c = copy(rho_c_new)
        Hr = copy(Hr_new)
        Cp_c = copy(Cp_c_new)
        alpha = copy(alpha_new)
        eta_s = copy(eta_s_new)
        eta_n = copy(eta_n_new)
        Tlast = copy(Tlast_new)
    
        # 2. Assemble and solve the stokes equations
        L,R = form_stokes(grid,eta_s,eta_n,rho_vx,rho_vy,bc,gx,gy,dt=dt)
        stokes_solution = L\R
        vx,vy,P = unpack(stokes_solution,grid;ghost=true)
    
        # Get the velocity at the cell centers:
        vxc,vyc = velocity_to_centers(grid,vx,vy)
        adiabatic_heating = compute_adiabatic_heating(grid,rho_c,Tlast,alpha,gx,gy,vxc,vyc)
        shear_heating = compute_shear_heating(grid,vx,vy,eta_n,eta_s)
        H = (adiabatic_heating .+ shear_heating .+ Hr).*0.0
    
        # 3. Compute the advection timestep:
        if itime > 1
            this_dtmax = min(1.2*dt,dtmax)
        else
            this_dtmax = dtmax
        end
        dt = compute_timestep(grid,vxc,vyc ; dtmax=this_dtmax)
        
        dTmax = Inf
        dTemp = nothing
        Tnew = nothing
        titer=1
        for titer=1:2# limit maximum temperature change
            # assemble and solve the energy equation
            println("Trying with timestep ",dt/3.15e7/1e6," Myr")
            L,R = assemble_energy_equation_center(grid,rho_c,Cp_c,kThermal,H,Tlast,dt,Tbcval);
            Tnew = L\R;
            Tnew = reshape(Tnew,grid.ny,grid.nx);
            Tnew = ghost_temperature_center(grid,Tnew,Tbcval);

            T = copy(Tnew)

            dTemp = Tnew-Tlast
            # compute the maximum temperature change
            dTmax = maximum(abs.(dTemp[2:end-1,2:end-1]))
            println("dTmax=",dTmax," dt=",dt/3.15e7/1e6)
            dt = min(dt,dTmax < 10.0 ? dt : dt*10.0/dTmax)
            if dTmax < 10.0
                break
            end
        end

        dT_subgrid_node = subgrid_temperature_relaxation_center!(markers,grid,dTemp,Cp_c[1,1],kThermal[1,1],dt)
        dT_remaining = dTemp - dT_subgrid_node

        cell_center_change_to_markers!(markers,grid,dT_remaining,"T")

        if time == 0.0 || time - last_plot >= plot_interval
            last_plot = time 
    
            plume_plots(grid,vx,vy,rho_c,Tnew)
    
            name = @sprintf("%s/viz.%04d.vtr",output_dir,iout)
            println("Writing visualization fle ",name)
            vn = velocity_to_basic_nodes(grid,vxc,vyc)

        visualization(grid,rho_c,eta_s,vn,P,Tnew[2:end-1,2:end-1],time/seconds_in_year;filename=name)
            name = @sprintf("%s/markers.%04d.vtp",output_dir,iout)
            visualization(markers,time/seconds_in_year;filename=name)
            iout += 1
        end
        
        println("Min/Max velocity: ",minimum(vyc)," ",maximum(vyc))
        move_markers_rk4!(markers,grid,vx,vy,dt,continuity_weight=1/3)
        time += dt
        println("Finished Step ",itime," time=",time/seconds_in_year/1e6," Myr")
     end
     return grid,markers,vxc,vyc,rho_c,eta_n,Tnew,time
 end
 @time grid,markers,vxc,vyc,rho_c,eta_n,Tnew,time = plume_model(1000);


Creating Markers...
  0.007587 seconds (19 allocations: 39.978 MiB)
Initial condition...
  0.659261 seconds (11.24 M allocations: 172.512 MiB, 7.35% gc time)
Trying with timestep 0.01838729602446387 Myr
dTmax=0.20876268684094157 dt=0.01838729602446387
Writing visualization fle plume/viz.0000.vtr
Min/Max velocity: -2.428237481343123e-9 1.8065118085314318e-9
Finished Step 1 time=0.01838729602446387 Myr
Trying with timestep 0.005491885181713079 Myr
dTmax=0.06319533328269245 dt=0.005491885181713079
Min/Max velocity: -2.154414654607241e-8 1.011789879045463e-8
Finished Step 2 time=0.023879181206176953 Myr
Trying with timestep 0.006590262218055694 Myr
dTmax=0.08240718548950099 dt=0.006590262218055694
Min/Max velocity: -1.1357440459509082e-8 3.806517112910495e-9
Finished Step 3 time=0.030469443424232644 Myr
Trying with timestep 0.007311875631164393 Myr
dTmax=0.09841559427286484 dt=0.007311875631164393
Min/Max velocity: -1.7585177858440868e-8 6.036705045665278e-9
Finished Step 4 time=0.03778131

Trying with timestep 0.0031090814951961663 Myr
dTmax=0.8434308312475878 dt=0.0031090814951961663
Min/Max velocity: -3.60919622379981e-8 2.001312106811623e-8
Finished Step 50 time=0.1714430700223891 Myr
Trying with timestep 0.0030249102897487654 Myr
dTmax=0.8290022788214628 dt=0.0030249102897487654
Min/Max velocity: -3.666297288407462e-8 1.987191503698062e-8
Finished Step 51 time=0.17446798031213784 Myr
Trying with timestep 0.002971957966812225 Myr
dTmax=0.8230532135989961 dt=0.002971957966812225
Min/Max velocity: -3.73411034308513e-8 1.9703734749301085e-8
Finished Step 52 time=0.17743993827895005 Myr
Trying with timestep 0.002941656030420117 Myr
dTmax=0.8196028267389011 dt=0.002941656030420117
Min/Max velocity: -4.0901035457091905e-8 1.9524858395585124e-8
Finished Step 53 time=0.18038159430937015 Myr
Trying with timestep 0.0029311867473593824 Myr
dTmax=0.8224588947291522 dt=0.0029311867473593824
Min/Max velocity: -4.3739517872141096e-8 1.931090034949866e-8
Finished Step 54 time=0.18331

Min/Max velocity: -3.158804950082497e-8 1.01286092059792e-8
Finished Step 100 time=0.3398412164514018 Myr
Trying with timestep 0.0038321545077798854 Myr
dTmax=1.2897215217067242 dt=0.0038321545077798854
Min/Max velocity: -2.8897901157947415e-8 9.985929380739127e-9
Finished Step 101 time=0.34367337095918177 Myr
Trying with timestep 0.003989806863744582 Myr
dTmax=1.340882601708472 dt=0.003989806863744582
Min/Max velocity: -2.7739442282945106e-8 9.87783364356375e-9
Finished Step 102 time=0.34766317782292633 Myr
Trying with timestep 0.0041961677530191095 Myr
dTmax=1.4110170351973466 dt=0.0041961677530191095
Min/Max velocity: -2.6214638734536775e-8 9.763413163758483e-9
Finished Step 103 time=0.35185934557594545 Myr
Trying with timestep 0.004440691707734433 Myr
dTmax=1.4916861980927933 dt=0.004440691707734433
Min/Max velocity: -2.4033899457352842e-8 9.642848751934424e-9
Finished Step 104 time=0.35630003728367987 Myr
Trying with timestep 0.004697233734717161 Myr
dTmax=1.575240085144742 dt=0.0

dTmax=3.493255008365395 dt=0.008751014887411928
Min/Max velocity: -1.4547899750679386e-8 4.8364020105071254e-9
Finished Step 151 time=0.7167278166495662 Myr
Trying with timestep 0.008431526236690539 Myr
dTmax=3.4061609450077412 dt=0.008431526236690539
Min/Max velocity: -1.3969809621884791e-8 4.7112979542443214e-9
Finished Step 152 time=0.7251593428862567 Myr
Trying with timestep 0.00813087737237685 Myr
dTmax=3.288073193748005 dt=0.00813087737237685
Min/Max velocity: -1.385047735399305e-8 4.688976417841383e-9
Finished Step 153 time=0.7332902202586336 Myr
Trying with timestep 0.007834748433978238 Myr
dTmax=3.180448766873724 dt=0.007834748433978238
Min/Max velocity: -1.412553950280912e-8 4.697333993030267e-9
Finished Step 154 time=0.7411249686926119 Myr
Trying with timestep 0.007464922701586887 Myr
dTmax=3.0610009142924355 dt=0.007464922701586887
Min/Max velocity: -1.4346908575853446e-8 4.724052389275634e-9
Finished Step 155 time=0.7485898913941987 Myr
Trying with timestep 0.0071735955116

Finished Step 201 time=0.9581097919140059 Myr
Trying with timestep 0.0034073175784279067 Myr
dTmax=1.6731539349179911 dt=0.0034073175784279067
Min/Max velocity: -2.1529889287518282e-8 5.488817220230436e-9
Finished Step 202 time=0.9615171094924337 Myr
Trying with timestep 0.0034350291993088264 Myr
dTmax=1.6943528362844518 dt=0.0034350291993088264
Min/Max velocity: -2.1497657071393375e-8 5.515693375450229e-9
Finished Step 203 time=0.9649521386917426 Myr
Trying with timestep 0.0034483420100519346 Myr
dTmax=1.7106391936113141 dt=0.0034483420100519346
Min/Max velocity: -2.139217959882118e-8 5.538375539610704e-9
Finished Step 204 time=0.9684004807017945 Myr
Trying with timestep 0.003461020899789861 Myr
dTmax=1.7236341231478036 dt=0.003461020899789861
Min/Max velocity: -2.16758711141108e-8 5.563789024548907e-9
Finished Step 205 time=0.9718615016015844 Myr
Trying with timestep 0.0034910180261600375 Myr
dTmax=1.7467212508754528 dt=0.0034910180261600375
Min/Max velocity: -2.1952188480837953e-8 5

...

Finished Step 708 time=6.355765848051052 Myr
Trying with timestep 0.014455185776764804 Myr
dTmax=2.6285519302273315 dt=0.014455185776764804
Min/Max velocity: -1.101951024357036e-8 5.284329692283653e-9
Finished Step 709 time=6.370221033827817 Myr
Trying with timestep 0.014461140515913509 Myr
dTmax=3.1261149719525747 dt=0.014461140515913509
Min/Max velocity: -1.1014972682444062e-8 5.283317739833513e-9
Finished Step 710 time=6.384682174343731 Myr
Trying with timestep 0.014459269526382124 Myr
dTmax=3.257444347001183 dt=0.014459269526382124
Min/Max velocity: -1.1016397989479092e-8 5.283681721770657e-9
Finished Step 711 time=6.399141443870113 Myr
Trying with timestep 0.014466994177698528 Myr
dTmax=2.3976454889416345 dt=0.014466994177698528
Min/Max velocity: -1.1010515783943774e-8 5.281382907791106e-9
Finished Step 712 time=6.413608438047811 Myr
Trying with timestep 0.014471093264596021 Myr
dTmax=3.116496368180833 dt=0.014471093264596021
Min/Max velocity: -1.1007396941423767e-8 5.281499321907

Min/Max velocity: -1.0909418897837781e-8 5.230247398029572e-9
Finished Step 759 time=7.0969679833690025 Myr
Trying with timestep 0.01460005332280648 Myr
dTmax=2.896121679604221 dt=0.01460005332280648
Min/Max velocity: -1.0910170272525632e-8 5.231494028112151e-9
Finished Step 760 time=7.111568036691808 Myr
Trying with timestep 0.014621903125963957 Myr
dTmax=2.3220623565953247 dt=0.014621903125963957
Min/Max velocity: -1.0893866986228656e-8 5.2281635598874096e-9
Finished Step 761 time=7.126189939817773 Myr
Trying with timestep 0.01463040870337143 Myr
dTmax=2.955827385289922 dt=0.01463040870337143
Min/Max velocity: -1.0887533695696811e-8 5.2272491595759096e-9
Finished Step 762 time=7.140820348521144 Myr
Trying with timestep 0.01463909805638399 Myr
dTmax=2.9818568570199204 dt=0.01463909805638399
Min/Max velocity: -1.0881071164784479e-8 5.2276101267558405e-9
Finished Step 763 time=7.155459446577527 Myr
Trying with timestep 0.01466720907540115 Myr
dTmax=2.2159728665753846 dt=0.01466720907540

Min/Max velocity: -1.0751150801593451e-8 5.154764940553722e-9
Finished Step 810 time=7.847629950358628 Myr
Trying with timestep 0.014830645877397115 Myr
dTmax=2.9483596777813545 dt=0.014830645877397115
Min/Max velocity: -1.074053477215969e-8 5.151093532604724e-9
Finished Step 811 time=7.862460596236024 Myr
Trying with timestep 0.014859561077470893 Myr
dTmax=2.893525049812979 dt=0.014859561077470893
Min/Max velocity: -1.0719634779877467e-8 5.147719800898981e-9
Finished Step 812 time=7.877320157313495 Myr
Trying with timestep 0.014875664618674605 Myr
dTmax=3.4191165373372314 dt=0.014875664618674605
Min/Max velocity: -1.07080303181751e-8 5.1476173767335365e-9
Finished Step 813 time=7.8921958219321695 Myr
Trying with timestep 0.014888195541498709 Myr
dTmax=2.991851592374587 dt=0.014888195541498709
Min/Max velocity: -1.0699017708074624e-8 5.143710910391774e-9
Finished Step 814 time=7.907084017473669 Myr
Trying with timestep 0.014900290037945425 Myr
dTmax=3.1101269128787408 dt=0.014900290037

Min/Max velocity: -1.0137440331342922e-8 4.9425419536016176e-9
Finished Step 861 time=8.621471381388714 Myr
Trying with timestep 0.015739724515215613 Myr
dTmax=3.1824931336841473 dt=0.015739724515215613
Min/Max velocity: -1.0120194135913077e-8 4.934158799838642e-9
Finished Step 862 time=8.637211105903932 Myr
Trying with timestep 0.015777106006129908 Myr
dTmax=2.816737487827595 dt=0.015777106006129908
Min/Max velocity: -1.0096215850859054e-8 4.924689875166751e-9
Finished Step 863 time=8.65298821191006 Myr
Trying with timestep 0.015803578746590927 Myr
dTmax=3.4445823284168 dt=0.015803578746590927
Min/Max velocity: -1.0079303573827127e-8 4.919394586778566e-9
Finished Step 864 time=8.668791790656652 Myr
Trying with timestep 0.015843000422157156 Myr
dTmax=3.109258990242779 dt=0.015843000422157156
Min/Max velocity: -1.0054223536912828e-8 4.910158442545739e-9
Finished Step 865 time=8.684634791078809 Myr
Trying with timestep 0.01588007890842296 Myr
dTmax=2.8051767982396996 dt=0.015880078908422

Min/Max velocity: -8.365616675469627e-9 4.226292164076509e-9
Finished Step 912 time=9.489517619758798 Myr
Trying with timestep 0.01923456630747468 Myr
dTmax=4.125086750020273 dt=0.01923456630747468
Min/Max velocity: -8.281396377409857e-9 4.2034354622259564e-9
Finished Step 913 time=9.508752186066271 Myr
Trying with timestep 0.019428929980021794 Myr
dTmax=4.247205266212404 dt=0.019428929980021794
Min/Max velocity: -8.198550712960665e-9 4.179796088021799e-9
Finished Step 914 time=9.528181116046294 Myr
Trying with timestep 0.01963000890977988 Myr
dTmax=4.340370350028479 dt=0.01963000890977988
Min/Max velocity: -8.114569304164246e-9 4.158140944639207e-9
Finished Step 915 time=9.547811124956073 Myr
Trying with timestep 0.019846045540925304 Myr
dTmax=4.446614353317273 dt=0.019846045540925304
Min/Max velocity: -8.02623713682881e-9 4.133743939247255e-9
Finished Step 916 time=9.567657170496998 Myr
Trying with timestep 0.020071748179359365 Myr
dTmax=4.575476318548226 dt=0.020071748179359365
Min/

In [ ]:
# figure()
# # title(print(time/3.15e7))
# # pcolor(grid.xc,grid.yc,eta_n)
# gca().invert_yaxis()
# colorbar()
# show()

figure()
scatter(markers.x[1,:],markers.x[2,:],c=(markers.integers[markers.integerFields["material"],:]),s=0.1)
gca().invert_yaxis()
gca().set_aspect("equal")
colorbar()
show()

figure()
scatter(markers.x[1,:],markers.x[2,:],c=(markers.scalars[markers.scalarFields["rho"],:]),s=0.1)
gca().invert_yaxis()
gca().set_aspect("equal")
colorbar()
show()


figure()
scatter(markers.x[1,:],markers.x[2,:],c=(markers.scalars[markers.scalarFields["T"],:]),s=0.1)
gca().invert_yaxis()
gca().set_aspect("equal")
colorbar()
show()

In [79]:
# Optionally, run a few steps with the profiler
# using ProfileSVG
# @profview plume_model(10)